### Procédure de test et fonctionnement de `deviation.py`

**`1. Tests des Fonctions Utilitaires`**

| ID  | Fonctionnalité                 | Procédure                                               | Données de test                      | Résultat attendu                                                                       |
| :-- | :----------------------------- | :------------------------------------------------------ | :----------------------------------- | :------------------------------------------------------------------------------------- |
| U1  | `calcul_champ_electrique` valide | Appeler `calcul_champ_electrique(1e-9, 0.01)`             | charge=1e-9 C, surface=0.01 m²       | Retourne une valeur float positive (approx. 5645 V/m).                               |
| U2  | `calcul_champ_electrique` surf=0 | Appeler `calcul_champ_electrique(1e-9, 0)`              | charge=1e-9 C, surface=0 m²          | Lève une `ValueError` avec message "La surface ne peut être nulle ou négative".      |
| U3  | `calcul_champ_electrique` surf<0 | Appeler `calcul_champ_electrique(1e-9, -0.01)`            | charge=1e-9 C, surface=-0.01 m²      | Lève une `ValueError` avec message "La surface ne peut être nulle ou négative".      |
| U4  | `champ_electrique_v2` valide   | Appeler `champ_electrique_v2(0.1, 5000)`                | distance=0.1 m, potentiel=5000 V   | Retourne 50000.0.                                                                      |
| U5  | `champ_electrique_v2` dist=0   | Appeler `champ_electrique_v2(0, 5000)`                  | distance=0 m, potentiel=5000 V     | Lève une `ValueError` avec message "La distance doit être positive.".                 |
| U6  | `champ_electrique_v2` dist<0   | Appeler `champ_electrique_v2(-0.1, 5000)`               | distance=-0.1 m, potentiel=5000 V  | Lève une `ValueError` avec message "La distance doit être positive.".                 |

**`2. Tests de la Classe `particule` (Initialisation et Calculs)`**

| ID  | Fonctionnalité                     | Procédure                                                                    | Données de test                                                     | Résultat attendu                                                                                                                               |
| :-- | :--------------------------------- | :--------------------------------------------------------------------------- | :------------------------------------------------------------------ | :--------------------------------------------------------------------------------------------------------------------------------------------- |
| P1  | Initialisation valide              | `p = particule((1, 1), v_initiale=1e5, angle_initial=np.pi/4, hauteur_initiale=0.1)` | m=1u, c=1e, v0=1e5, angle=pi/4, h=0.1                             | Objet `p` créé. `p.m=1`, `p.c=1`, `p.vo=1e5`, `p.angle=pi/4`, `p.height=0.1`. `p.mq` calculé (approx 1.036e-8 kg/C). `is_incertitude=False`. |
| P2  | Initialisation charge nulle        | Tenter `p = particule((1, 0))`                                                 | m=1u, c=0e                                                          | Lève une `ValueError` avec message "La charge de la particule ne peut pas être nulle.".                                                        |
| P3  | Initialisation incertitude       | `p = particule((1.1, 0.9), ..., is_incertitude=True, incertitude_unique=True, base_mq=(1,1))` | m=1.1u, c=0.9e, is_incertitude=True, unique=True, base=(1,1) | Objet `p` créé. `p.is_incertitude=True`, `p.incertitude_unique=True`, `p.base_mq=(1,1)`.                                                     |
| P4  | `equation_trajectoire` (cas simple)| `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.equation_trajectoire(0.01, -50000)` | E=-50kV/m (calculé depuis V=-5kV, d=0.1m), x=0.01m                | Retourne une valeur `y` float (calcul analytique à vérifier, devrait être < 0.1).                                                            |
| P5  | `trajectoire`                    | `p = particule(...)`. Calculer `x, y = p.trajectoire(E, 0, 0.1, n_points=100)` | E, x_min=0, x_max=0.1, n=100                                      | Retourne deux `np.ndarray` `x` et `y`, chacun de taille 100. `x` va de 0 à 0.1.                                                                 |
| P6  | `point_contact` (contact attendu)  | `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.point_contact(-50000)` | E = -50000 V/m                                                  | Retourne une valeur `x` float positive (calcul analytique à vérifier).                                                                        |
| P7  | `point_contact` (pas de contact)   | `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.point_contact(1000)`   | E = +1000 V/m (champ repulsif)                                  | Retourne `None`.                                                                                                                               |
| P8  | `point_contact` (E=0)              | `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.point_contact(0)`    | E = 0 V/m                                                         | Retourne `h * tan(angle)` (approx 0.1 * 1 = 0.1).                                                                                              |
| P9  | `angle_incident` (contact attendu) | `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.angle_incident(-50000)` | E = -50000 V/m                                                  | Retourne un angle float en radians (calcul analytique à vérifier, devrait être > pi/2 si l'axe y est vers le haut).                            |
| P10 | `angle_incident` (pas de contact)  | `p = particule((1, 1), v0=1e5, angle=pi/4, h=0.1)`. Calculer `p.angle_incident(1000)`  | E = +1000 V/m                                                   | Retourne `None` (car `point_contact` retourne `None`).                                                                                         |

**`3. Tests des Fonctions de Tracé (Statique - Multiples Particules)`**

| ID  | Fonctionnalité                             | Procédure                                                                                                  | Données de test                                                            | Résultat attendu (si `create_plot=False`)                                                                                               |
| :-- | :----------------------------------------- | :--------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------- |
| T1  | `tracer_ensemble_trajectoires` valide      | `fig, ax = plt.subplots(); tracer_ensemble_trajectoires([(1,1),(2,1)], 1e5, -5000, ax=ax, create_plot=False)` | Particules (1,1), (2,1), V0=1e5, Pot=-5000                               | L'objet `ax` est modifié : contient 2 lignes (trajectoires), 1 ligne (échantillon), légende, texte des angles. Retourne `ax`. Pas d'erreur. |
| T2  | `tracer_ensemble_trajectoires` charge inv. | Tenter `tracer_ensemble_trajectoires([(1,1),(2,-1)], ...)`                                                    | Charges +1 et -1                                                           | Lève `ValueError` avec message "Les charges...même signe...".                                                                       |
| T3  | `tracer_ensemble_trajectoires` pas contact | `tracer_ensemble_trajectoires([(1,1)], 1e5, 100, ax=ax, create_plot=False)`                                    | Pot=+100 (répulsif)                                                        | `ax` modifié. Trajectoire tracée jusqu'à la limite x du graphe. Texte indique "Pas de contact".                                      |
| T4  | `create_particules_incertitudes` (qE<0)    | `p=particule((1,1),...); inc={'m':0.1,'q':0.1,...}; create_particules_incertitudes([p], inc, -50000)`          | Particule (1,1), E<0, incertitudes 10%                                   | Retourne (liste[p, p_min, p_max], E_min, E_max). Vérifier params de p_min/p_max et E_min/E_max selon la logique qE<0.                  |
| T5  | `create_particules_incertitudes` (qE>0)    | `p=particule((1,-1),...); inc={...}; create_particules_incertitudes([p], inc, -50000)`                       | Particule (1,-1), E<0 => qE>0                                            | Retourne (liste[p, p_min, p_max], E_min, E_max). Vérifier params de p_min/p_max et E_min/E_max selon la logique qE>=0.                 |
| T6  | `tracer_ens_traj_avec_incertitudes` valide | `fig,ax=plt.subplots(); tracer_ens...([(1,1)], 1e5, inc, -5000, ax=ax, create_plot=False)`                   | Particule (1,1), incertitudes valides, Pot=-5000                           | `ax` modifié. Contient 1 ligne pleine (nominal), 2 lignes pointillées (incert.). Couleurs cohérentes. Label incertitude présent.       |
| T7  | `tracer_ens_traj_avec_incertitudes` chg inv| Tenter `tracer_ens...([(1,1),(2,-1)], ...)`                                                                 | Charges +1 et -1                                                           | Lève `ValueError` avec message "Les charges...même signe...".                                                                       |

**`4. Tests des Fonctions de Tracé (Statique - Une Particule, Multiples Potentiels)`**

| ID  | Fonctionnalité                                          | Procédure                                                                                                      | Données de test                                                             | Résultat attendu (si `create_plot=False`)                                                                                             |
| :-- | :------------------------------------------------------ | :------------------------------------------------------------------------------------------------------------- | :-------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------ |
| P1  | `tracer_ensemble_potentiels` valide                     | `fig,ax=plt.subplots(); tracer_ensemble_potentiels((1,1), 1e5, [-1k, -5k], ax=ax, create_plot=False)`           | Particule (1,1), V0=1e5, Pots=[-1000, -5000]                               | `ax` modifié. Contient 2 lignes (une par potentiel), 1 ligne échantillon, légende avec labels "Trajectoire à -1000 V", etc. Texte angles OK. |
| P2  | `tracer_ensemble_potentiels` charge nulle               | Tenter `tracer_ensemble_potentiels((1,0), ...)`                                                                  | Charge=0                                                                    | Lève `ValueError` (levée par le constructeur `particule`).                                                                          |
| P3  | `tracer_ens_traj_potentiels_avec_incertitudes` valide   | `fig,ax=plt.subplots(); tracer_ens...((1,1), 1e5, inc, [-1k, -5k], ax=ax, create_plot=False)`                    | Particule (1,1), incertitudes, Pots=[-1000, -5000]                          | `ax` modifié. Contient 2*3=6 lignes (1 nominale + 2 incert. par potentiel). Labels et couleurs corrects. Texte angles OK.             |
| P4  | `tracer_ens_traj_potentiels_avec_incertitudes` chg nulle| Tenter `tracer_ens...((1,0), ...)`                                                                               | Charge=0                                                                    | Lève `ValueError` (levée par le constructeur `particule`).                                                                          |

**`5. Tests de la Fonction de Tracé (Dynamique)`**

| ID  | Fonctionnalité                               | Procédure                                                                                                | Données de test                                                              | Résultat attendu                                                                                                                               |
| :-- | :------------------------------------------- | :------------------------------------------------------------------------------------------------------- | :--------------------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------------- |
| D1  | `tracer_ensemble_trajectoires_dynamique` run | Appeler `tracer_ensemble_trajectoires_dynamique([(1,1),(2,1)], 1e5, -5k, 5k)`                             | Particules (1,1), (2,1), V0=1e5, Pot range [-5k, 5k]                         | Une fenêtre Matplotlib s'ouvre SANS erreur. Contient un graphique initial et des sliders (Potentiel, Zoom).                                     |
| D2  | `tracer_ensemble_trajectoires_dynamique` chg inv | Tenter `tracer_ensemble_trajectoires_dynamique([(1,1),(2,-1)], ...)`                                      | Charges +1 et -1                                                             | Lève `ValueError` avec message "Les charges...même signe..." lors de l'appel initial. La fenêtre ne devrait pas s'ouvrir ou se fermer rapidement. |
| D3  | Interaction Slider Potentiel (Manuel)        | Lancer D1. Déplacer le slider "Potentiel (V)".                                                         | Interaction utilisateur                                                      | Le graphique dans la fenêtre se met à jour dynamiquement pour refléter le nouveau potentiel, sans erreur. Le texte des angles est mis à jour. |
| D4  | Interaction Slider Zoom (Manuel)             | Lancer D1. Déplacer le slider "Zoom".                                                                  | Interaction utilisateur                                                      | Les limites x et y du graphique sont modifiées dynamiquement, zoomant sur la zone de contact (si contact il y a), sans erreur.            |

*Note : Les tests des fonctions de tracé (marqués T, P, D) avec `create_plot=False` nécessitent la création manuelle d'une figure et d'un axe Matplotlib (`fig, ax = plt.subplots()`) avant l'appel. Les tests marqués "(Manuel)" pour les sliders nécessitent une exécution interactive du script.*